# Pre-calculate the adjacency matrix and the ProteinGCN embeddings

In [1]:
%load_ext memory_profiler
import os, random, math, argparse
import torch

from model_ABDG_v2 import ProteinGCN
# from data import 
from data import ProteinDataset,ProteinDockingDataset, get_train_val_test_loader,collate_pool, collate_pool_docking
from utils import randomSeed
import config as cfg
import numpy as np
import tqdm

In [2]:
import prody as pd
pd.confProDy(verbosity='none')
import dgl
from scipy.spatial import KDTree

# How to make the adjacencies file
# For a given PDB pair
#[(Ligand), (Receptor)]
# [(356, 45, 44, 'A'), (730, 96, 91, 'A')]
# (atom index, residue index from pdb, unique residue number across chains, chain letter)
def find_adjacencies(pdb_l,pdb_r, dist_cutoff=4.5):    
    geometry_l = pdb_l.getCoords()
    elements_l = pdb_l.getNames()
    residues_l = pdb_l.getResnums()
    #Chains are characters, 'A','B' etc., just convert to index 0,1,2...
    chains_l = pdb_l.getChids()
    _,chains_idx_l = np.unique(pdb_l.getChids(),return_inverse=True)    
    #Use the chains to make the residues unique
    reschain_l = np.transpose([residues_l,chains_idx_l]) #Make Nx2 vector
    #Note the 0-indexed residues here
    _,residues_idx_l = np.unique(reschain_l,axis=0,return_inverse=True)
    #Residues_l is now a zero-indexed vector of all unique residues
     
    geometry_r = pdb_r.getCoords()
    elements_r = pdb_r.getNames()
    residues_r = pdb_r.getResnums()
    chains_r = pdb_r.getChids()
    _,chains_idx_r = np.unique(pdb_r.getChids(),return_inverse=True)    
    #Use the chains to make the residues unique
    reschain_r = np.transpose([residues_r,chains_idx_r]) #Make Nx2 vector
    _,residues_idx_r = np.unique(reschain_r,axis=0,return_inverse=True)
    #Residues_idx_l is now a zero-indexed vector of all unique residues
    
#     top_residue_chain = 0
#     for i in np.arange(1,residues_r.shape[0]):
#         if not chains_r[i] == chains_r[i-1]:
#             top_residue_chain = residues_r[i-1]
#         residues_r[i] = residues_r[i]+ top_residue_chain
        
    combo = np.concatenate( (geometry_l,geometry_r),axis=0)
    r_idx_min = geometry_l.shape[0] #any node index with this or higher is an R
    kdtree = KDTree(combo)
    neighborhood = kdtree.query_ball_point(geometry_l, r=dist_cutoff) 
    
    # To later populate some sort of adjacency matrix, we'll create a tuple of the form
    # [(ligand atom index, ligand residue, ligand chain), (receptor atom index, receptor residue, receptor chain)]
    # This can later be processed into any sort of adjacency matrix or mask

    #Get the list of all l atoms that are within 4.5A of the R atoms
    in_interface_l = np.where([any(np.array(f)>=r_idx_min) for f in neighborhood])[0]
    #print(f"Found {len(in_interface_l)} atoms that are near R")
    adjacencies = []
    for l_idx in in_interface_l: # l_idx = ligand atom index
        #Get the local neighborhood as defined by the threshold
        local_n = np.array(neighborhood[l_idx])
        #Get the receptor index
        indices_r = local_n[np.where(local_n>=r_idx_min)] - r_idx_min
        #Create the list of tuples to be analyzed later
        for r_idx in indices_r:
#             print(l_idx,r_idx, residues_r.shape,residues_l.shape)
            adjacencies.append([(l_idx,residues_l[l_idx],residues_idx_l[l_idx],chains_l[l_idx]), (r_idx,residues_r[r_idx],residues_idx_r[r_idx],chains_r[r_idx])])
    return adjacencies

#Get list of all directories in the DB5 Root
DB5ROOT_dir = "/mnt/disks/amanda200/DB5/raw/"
protein_dirs = os.listdir(DB5ROOT_dir)
#Because we want ground truth information, get the bound confirm

all_dirs = []
for directory in protein_dirs:
    if not "DS_Store" in directory:
        all_dirs.append(directory)

#For DB5, all_dirs are the PDBIDS, ie, 2A9K
PDBIDs = all_dirs

# for PDBID in PDBIDs:

#     pdb_path_l = os.path.join(DB5ROOT_dir,PDBID,f'{PDBID}_l_b_cleaned.pdb')
#     pdb_path_r = os.path.join(DB5ROOT_dir,PDBID,f'{PDBID}_r_b_cleaned.pdb')
    
#     pdb_l = pd.parsePDB(pdb_path_l)
#     pdb_r = pd.parsePDB(pdb_path_r)    
    
#     adjacencies = find_adjacencies(pdb_l,pdb_r)
#     savepath = os.path.join(DB5ROOT_dir,PDBID,f'{PDBID}_b_adjacencies.npy')
#     np.save(savepath,adjacencies)
#     print(f"Saved file: {savepath} which had {len(adjacencies)} entries")

Using backend: pytorch


In [4]:
args = {'name': 'abdg_demo',
            'pkl_dir': '/home/ambeck/6.883ProteinDocking/data/firstDB5try/',
            'pdb_dir': '/mnt/disks/amanda200/DB5/raw/',
            'protein_dir': '/mnt/disks/amanda200/DB5/raw/',
            'useDB5Bound':True,
            'save_dir': './data/pkl/results/',
            'id_prop': 'protein_id_prop.csv',
            'atom_init': 'protein_atom_init.json',
            'pretrained': './pretrained/pretrained.pth.tar',
            'avg_sample': 500,
            'seed': 1234,
            'epochs': 0,
            'batch_size': 1,
            'train': 0.0,
            'val': 0.0,
            'test': 1.0,
            'testing': False,
            'lr': 0.001, 'h_a': 64, 'h_g': 32,
            'n_conv': 4, 'save_checkpoints': True,
            'print_freq': 10, 
            'workers': 1,
           }
print('Torch Device being used: ', cfg.device)

# create the savepath
savepath = args["save_dir"] + str(args["name"]) + '/'
if not os.path.exists(savepath):
    os.makedirs(savepath)

randomSeed(args["seed"])

# create train/val/test dataset separately
assert os.path.exists(args["protein_dir"]), '{} does not exist!'.format(args["protein_dir"])
dirs_label = [d[:10] for d in os.listdir(args["pkl_dir"]) if not d.startswith('.DS_Store')]
# all_dirs = [d for d in os.listdir(args["protein_dir"]) if not d.startswith('.DS_Store')]
base_dir=set(dirs_label)
dir_r = []
dir_l = []
dir_r.extend(d+'r_u_cleane.pkl' for d in base_dir)
dir_l.extend(d+'l_u_cleane.pkl' for d in base_dir)
all_dirs = []
for r,l in zip(dir_r, dir_l):
    all_dirs.append(r)
    all_dirs.append(l)

dir_len = len(all_dirs)
indices = list(range(dir_len))
random.shuffle(indices)

train_size = math.floor(args["train"] * dir_len)
val_size = math.floor(args["val"] * dir_len)
test_size = math.floor(args["test"] * dir_len)

if val_size == 0:
    print(
        'No protein directory given for validation!! Please recheck the split ratios, ignore if this is intended.')
if test_size == 0:
    print('No protein directory given for testing!! Please recheck the split ratios, ignore if this is intended.')

test_dirs = all_dirs[:test_size]
train_dirs = all_dirs[test_size:test_size + train_size]
val_dirs = all_dirs[test_size + train_size:test_size + train_size + val_size]
print('Testing on {} protein directories:'.format(len(test_dirs)))

def loadProteinDataSetAndModel():

    dataset = ProteinDataset(args["pkl_dir"], args["id_prop"], args["atom_init"], random_seed=args["seed"])
#     dataset = ProteinDockingDataset(args["pkl_dir"],args["pdb_dir"], args["id_prop"],args['useDB5Bound'], args["atom_init"])
    print('Dataset length: ', len(dataset))

    # load all model args from pretrained model
    if args["pretrained"] is not None and os.path.isfile(args["pretrained"]):
        print("=> loading model params '{}'".format(args["pretrained"]))
        model_checkpoint = torch.load(args["pretrained"], map_location=lambda storage, loc: storage)
        model_args = argparse.Namespace(**model_checkpoint['args'])
        # override all args value with model_args
        args["h_a"] = model_args.h_a
        args["h_g"] = model_args.h_g
        args["n_conv"] = model_args.n_conv
        args["random_seed"] = model_args.seed
        args["lr"] = model_args.lr

        print("=> loaded model params '{}'".format(args["pretrained"]))
    else:
        print("=> no model params found at '{}'".format(args["pretrained"]))

    args["random_seed"] = args["seed"]
    structures, _, _ = dataset[0] #Getting the size from the ligand
    h_b = structures[1].shape[-1] 
    args['h_b'] = h_b  # Dim of the bond embedding initialization

    # Use DataParallel for faster training
    print("Let's use", torch.cuda.device_count(), "GPUs and Data Parallel Model.")
    # print(kwargs)
    model = ProteinGCN(**args)
    return model, dataset


Torch Device being used:  cpu
No protein directory given for validation!! Please recheck the split ratios, ignore if this is intended.
Testing on 464 protein directories:


In [5]:
# dataset = ProteinDockingDataset(args["pkl_dir"],args["pdb_dir"], args["id_prop"],args['useDB5Bound'], args["atom_init"])
dataset = ProteinDataset(args["pkl_dir"], args["id_prop"],args["atom_init"])

In [7]:
model, dataset = loadProteinDataSetAndModel()

Dataset length:  920
=> loading model params './pretrained/pretrained.pth.tar'
=> loaded model params './pretrained/pretrained.pth.tar'
Let's use 0 GPUs and Data Parallel Model.


In [8]:
test_loader = get_train_val_test_loader(dataset, train_dirs, val_dirs, test_dirs,
                                                                      collate_fn    = collate_pool,
                                                                      num_workers   = args["workers"],
                                                                        batch_size    = args["batch_size"],
                                                                      pin_memory    = False,
                                                                      predict=True)

In [9]:
def getInputHACK(inputs):
    return [inputs[0], inputs[1], inputs[2], inputs[4], inputs[5]]

def loadAdjacencyMatrix(pdb,pdb_dir,chooseBound=True):
    #DB5 specific
    boundchar = 'b'
    if not chooseBound:
        boundchar = 'u'
    #The contents of the numpy file looks like this:
    #(437, 56, 55, 'A'), (520, 69, 65, 'A')
    #actually, they are all saved as strings due to some quirk with the saving
    adjacencies_full = np.load(os.path.join(pdb_dir, pdb, f'{pdb}_{boundchar}_adjacencies.npy'),allow_pickle=True)
    
    #Return just amino acids indices, which is not unique. 
    #Note that we're *not* using the residue index in the pdb but instead a 0-indexed unique ID
    adjacencies_short = [[int(a[0][2]), int(a[1][2])] for a in adjacencies_full]
    adjacencies_short = np.array(adjacencies_short)
    # Get only the unques
    adjacencies_short_unique = np.unique(adjacencies_short,axis=0)
#     print(adjacencies_short_unique)
    
    return adjacencies_short_unique



In [10]:
PDB_DIR = '/mnt/disks/amanda200/DB5/raw/'
OUTPUT_DIR = '/mnt/disks/amanda200/bounddb5_processed/'

max_amino_r = 0
max_amino_l = 0
for PDBID in PDBIDs:
    adjacencies = loadAdjacencyMatrix(PDBID,PDB_DIR)
#     torch.save(adjacencies,os.path.join(OUTPUT_DIR,'adjacencies',f"{PDBID}.pkl"))
    max_amino_r = max(max_amino_r,max(adjacencies[:,1]))
    max_amino_l = max(max_amino_l,max(adjacencies[:,0]))
    print(f"completed {PDBID} with size {adjacencies.shape}")
print(max_amino_l,max_amino_r)

completed 1JWH with size (32, 2)
completed 1EWY with size (36, 2)
completed 1AKJ with size (48, 2)
completed 1GXD with size (59, 2)
completed 2O3B with size (39, 2)
completed 2BTF with size (60, 2)
completed 2J0T with size (49, 2)
completed 1JK9 with size (68, 2)
completed 2GTP with size (45, 2)
completed 2NZ8 with size (63, 2)
completed 2JEL with size (47, 2)
completed 1IJK with size (35, 2)
completed 3F1P with size (47, 2)
completed 1FAK with size (84, 2)
completed 2HLE with size (69, 2)
completed 2PCC with size (25, 2)
completed 3EOA with size (34, 2)
completed 1GPW with size (54, 2)
completed 1E6J with size (42, 2)
completed 2W9E with size (44, 2)
completed 1IRA with size (84, 2)
completed 1EAW with size (61, 2)
completed 1US7 with size (34, 2)
completed 1OYV with size (71, 2)
completed 2OOR with size (56, 2)
completed 1M10 with size (56, 2)
completed 3SGQ with size (44, 2)
completed 3L5W with size (28, 2)
completed 2VDB with size (55, 2)
completed 3L89 with size (54, 2)
completed 

In [11]:

#Collate_pool is called to produce this out of the enumerate
#input_data = (final_protein_atom_fea, final_nbr_fea, final_nbr_fea_idx, None, final_atom_amino_idx, final_atom_mask)
#batch_data = (batch_protein_ids, np.concatenate(batch_amino_crystal))
#target tuples = (final_target, torch.cat(final_amino_target))

ligands_unembedded = {}
ligands_embedded = {}
receptors_unembedded = {}
receptors_embedded = {}
adjacencies = {}

PDB_DIR = '/mnt/disks/amanda200/DB5/raw/'
OUTPUT_DIR = '/mnt/disks/amanda200/bounddb5_processed/'


for protein_batch_iter, (input_data, batch_data, target_tuples) in enumerate(test_loader):
    print(f"{protein_batch_iter}: {batch_data[0]}")
    pdb = batch_data[0][0].split('_')[0]
    isligand = batch_data[0][0].split('_')[2]=='l'
    isbound = batch_data[0][0].split('_')[3]=='b'

    if not isbound: #Skipping unbound for now
        continue
    
    pgcn_data = getInputHACK(input_data)
    
    if isligand:
        torch.save(pgcn_data,os.path.join(OUTPUT_DIR,'ligand',f"{pdb}.pkl"))
    else:
        torch.save(pgcn_data,os.path.join(OUTPUT_DIR,'receptor',f"{pdb}.pkl"))
        
    amino_emb, protein_emb_temp = model(pgcn_data)
    
    if isligand:
        torch.save((amino_emb, protein_emb_temp),os.path.join(OUTPUT_DIR,'emb_ligand',f"{pdb}.pkl"))
    else:
        torch.save((amino_emb, protein_emb_temp),os.path.join(OUTPUT_DIR,'emb_receptor',f"{pdb}.pkl"))
    
    adjacencies = loadAdjacencyMatrix(pdb,PDB_DIR)
    torch.save(adjacencies,os.path.join(OUTPUT_DIR,'adjacencies',f"{pdb}.pkl"))



0: ['2I25_2I25_r_b_cleane']
Atom Mask Storage
0.003345489501953125


NameError: name 'barf' is not defined

In [12]:
print(isligand)
print(pgcn_data[2].size())
print(amino_emb.size())
print(protein_emb_temp.size())

False
torch.Size([1, 877, 50])
torch.Size([114, 64])
torch.Size([114, 32])


In [11]:
adjacencies

[[73, 170],
 [73, 170],
 [74, 192],
 [74, 171],
 [74, 171],
 [74, 192],
 [74, 171],
 [74, 171],
 [74, 192],
 [74, 192],
 [74, 170],
 [74, 171],
 [74, 170],
 [74, 170],
 [74, 170],
 [74, 170],
 [74, 170],
 [74, 192],
 [74, 192],
 [74, 170],
 [74, 168],
 [74, 170],
 [74, 170],
 [74, 192],
 [74, 192],
 [74, 168],
 [74, 168],
 [74, 170],
 [74, 170],
 [74, 168],
 [74, 170],
 [74, 170],
 [74, 170],
 [74, 170],
 [74, 170],
 [74, 170],
 [74, 168],
 [74, 170],
 [74, 170],
 [74, 168],
 [74, 168],
 [75, 170],
 [75, 170],
 [75, 170],
 [75, 171],
 [75, 171],
 [75, 209],
 [75, 170],
 [75, 170],
 [75, 170],
 [75, 170],
 [75, 170],
 [75, 170],
 [75, 171],
 [75, 209],
 [75, 171],
 [77, 231],
 [77, 231],
 [77, 231],
 [77, 231],
 [77, 231],
 [77, 231],
 [77, 192],
 [77, 192],
 [77, 153],
 [77, 153],
 [77, 153],
 [77, 192],
 [77, 192],
 [77, 191],
 [78, 231],
 [78, 231],
 [78, 231],
 [78, 231],
 [78, 231],
 [78, 231],
 [78, 231],
 [78, 231],
 [78, 231],
 [78, 231],
 [78, 210],
 [78, 210],
 [79, 231],
 [79

In [18]:
PDB_DIR = '/mnt/disks/amanda200/DB5/raw/'
OUTPUT_DIR = '/mnt/disks/amanda200/bounddb5_processed/'
pdb_list=[]
batch=0
for protein_batch_iter, (input_data, batch_data, target_tuples) in enumerate(test_loader):
    pdb = batch_data[0][0].split('_')[0]
    pdb_list.append(pdb)
    batch+=1
    print(batch)
np.save(pdb_list,os.path.join(OUTPUT_DIR, 'names.npy'))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


TypeError: expected str, bytes or os.PathLike object, not list